# Motivation

This notebooks serves a simple logfile patching purpose. As there are different ways to define the interval when a model
is the `most recent` model for a certain interval, we allow patching the logfile to the desired definition.

By default our pipeline assumes a model is most recent for the time AFTER the training interval.
However, sometimes we want to consider the model most recent for the time DURING the training interval.

This script mutates the `most_recent_model` field in the logfile to the non-default 
definition (during training interval).

In [ ]:
from modyn.supervisor.internal.pipeline_executor.models import PipelineLogs


from modyn.supervisor.internal.grpc.enums import PipelineStage
# fill missing times in cumulative plot


from analytics.app.data.transform import logs_dataframe
from pathlib import Path
from analytics.app.data.transform import dfs_models_and_evals


%load_ext autoreload
%autoreload 2

# Data loading

In [ ]:
# VARIABLES

pipeline_logfile = Path("/Users/robinholzinger/robin/dev/eth/modyn-2/.analytics.log/.data/pipeline_6o/pipeline.log")

In [ ]:
logs = PipelineLogs.model_validate_json(pipeline_logfile.read_text())


In [ ]:
trains = [(l_ for l_ in logs.supervisor_logs.stage_runs if l_.id == PipelineStage.HANDLE_SINGLE_TRIGGER.name)]
evals = [(l_ for l_ in logs.supervisor_logs.stage_runs if l_.id == PipelineStage.EVALUATE_SINGLE.name and l_.info.eval_request.dataset_id == "cglm_landmark_min25-test")]


In [ ]:
from analytics.app.data.transform import dfs_models_and_evals

df_logs = logs_dataframe(logs)
# max_timestamp = df_logs["sample_time"].max()
max_timestamp = df_logs["sample_time"].max()
df_models, eval_requests, evals_metrics = dfs_models_and_evals(logs, max_timestamp)

# Data exploration

In [ ]:
df_models

In [ ]:
eval_requests

In [ ]:
eval_requests[eval_requests["currently_active_model"]]

# Patch logfile

In [ ]:
for eval_log in logs.supervisor_logs.stage_runs:
    if eval_log.id == PipelineStage.EVALUATE_SINGLE.name:
        # Let's throw away all information about the most recent model, let's rebuild it
        eval_log.info.eval_request.currently_active_model = False

        # For a fixed interval the evaluation request of a certain model is the most recent, if the model training
        # interval center lies within the evaluation interval.
        # Note: this is not a generic solution, but works for the slicing case with fixed evaluation and trigger
        # intervals in the same order of magnitude.
        model_row = df_models[df_models["id_model"] == eval_log.info.eval_request.id_model]
        assert len(model_row) == 1

        training_center = (model_row.iloc[0]["train_start"].to_pydatetime().timestamp() + model_row.iloc[0]["train_end"].to_pydatetime().timestamp()) / 2
        eval_log.info.eval_request.currently_active_model = eval_log.info.eval_request.interval_start <= training_center <= eval_log.info.eval_request.interval_end
        eval_log.info.eval_request.currently_trained_model = eval_log.info.eval_request.interval_start <= training_center <= eval_log.info.eval_request.interval_end

In [ ]:
# Write results back
pipeline_logfile.write_text(logs.model_dump_json(by_alias=True))

In [ ]:

def patch_logfile(path):
    logs = PipelineLogs.model_validate_json(path.read_text())
    trains = [(l_ for l_ in logs.supervisor_logs.stage_runs if l_.id == PipelineStage.HANDLE_SINGLE_TRIGGER.name)]
    evals = [(l_ for l_ in logs.supervisor_logs.stage_runs if l_.id == PipelineStage.EVALUATE_SINGLE.name and l_.info.eval_request.dataset_id == "cglm_landmark_min25-test")]
    df_logs = logs_dataframe(logs)
    max_timestamp = df_logs["sample_time"].max()
    df_models, eval_requests, evals_metrics = dfs_models_and_evals(logs, max_timestamp)

    for eval_log in logs.supervisor_logs.stage_runs:
        if eval_log.id == PipelineStage.EVALUATE_SINGLE.name:
            # Let's throw away all information about the most recent model, let's rebuild it
            eval_log.info.eval_request.currently_active_model = False
    
            # For a fixed interval the evaluation request of a certain model is the most recent, if the model training
            # interval center lies within the evaluation interval.
            # Note: this is not a generic solution, but works for the slicing case with fixed evaluation and trigger
            # intervals in the same order of magnitude.
            model_row = df_models[df_models["id_model"] == eval_log.info.eval_request.id_model]
            assert len(model_row) == 1
    
            training_center = (model_row.iloc[0]["train_start"].to_pydatetime().timestamp() + model_row.iloc[0]["train_end"].to_pydatetime().timestamp()) / 2
            eval_log.info.eval_request.currently_active_model = eval_log.info.eval_request.interval_start <= training_center <= eval_log.info.eval_request.interval_end
            eval_log.info.eval_request.currently_trained_model = eval_log.info.eval_request.interval_start <= training_center <= eval_log.info.eval_request.interval_end

    patched_path = path.parent / "pipeline.patched"
    patched_path.write_text(logs.model_dump_json(by_alias=True))

In [ ]:
log_dir = Path("/Users/mboether/phd/dynamic-data/sigmod-data/cglm-landmark/data_selection_50%/logs")
logfiles = [logfile for logfile in log_dir.glob("**/pipeline.log") if (logfile.parent / "snapshot").exists()]

In [ ]:
from tqdm import tqdm
for logfile in tqdm(logfiles):
    patch_logfile(logfile)